# Pre-processing spectra to make easier to search

Idea: make one big spectrum that we will then tag masses with amino acid sequences

Flow: 
1. Load all spectra
2. Make a single large spectrum called `S`
3. Find the `max_length` peptide from the maximum mass
4. Load the database
5. For each protein `P` of the database
    1. For each kmer `k` of length `max_length`
        1. Calculate each individual spectrum for `(b+, b++, y+, y++)` (we call `ts`)
        2. For each `ts`:
            1. For each mass `m` of this `ts`:
                1. Binary search `S` for `m` plus/minus tolerance
                2. If the mass is found, add the `k(m)` to a dictionary to keep for later split by ion type
6. Build a MassDawg for both `b` and `y` kmers taken from the search
7. For each input spectrum:
    1. Search both the `b` and `y` MassDawgs for sequences 
    2. Make an alignment
                

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from pyteomics import fasta
from collections import defaultdict
from src.sequence.gen_spectra import gen_spectrum, gen_min_ordering
from src.file_io import mzML
from src.objects import Spectrum, Database
from src.utils import ppm_to_da
from src.scoring import scoring
from src.tree import Tree
from src.identfication import alignment

from src.database import extract_protein_name

import bisect

from mass_dawg import PyMassDawg

import numpy as np

ppm_tol = 20

In [2]:
def longest_array(window, length):
    return int(np.ceil(1/window * length * 186.079313))

def sparse_it(spectrum, window, length):
    sparse = np.zeros(longest_array(window, length))
    for mz in spectrum:
        mz_direct = int(mz/window)
        sparse[mz_direct] = 1
    
    return sparse

## 1. Load all spectra

In [3]:
spectra_file = '/Users/zacharymcgrath/Desktop/nod2 data/single/singleRealSpectrum.mzml'
fasta_file = '/Users/zacharymcgrath/Desktop/nod2 data/filteredNOD2.fasta'

In [4]:
spectra = mzML.read(spectra_file, peak_filter=25)

## 2. Make the single large spectrum

In [5]:
all_spectra = sorted([x for spectrum in spectra for x in spectrum.spectrum])

## 3. Find the max length possible peptide

In [6]:
max_len = int(np.ceil(all_spectra[-1]/ 57.021464))

## 4. Load the database into RAM

In [7]:
prots = {}
for i, entry in enumerate(fasta.read(fasta_file)):
    name = extract_protein_name(entry)
    prots[name] = entry

## 5. The search

In [8]:
%%time
def find_kmer_hits(kmer: str, prot_name: str) -> None:
    for ion in 'by':
        for charge in [1, 2]:
            spec = gen_spectrum(kmer, ion=ion, charge=charge)['spectrum']

            for c, mass in enumerate(spec):

                da_tol = ppm_to_da(mass, ppm_tol)
                lb = mass - da_tol
                ub = mass + da_tol

                beginning_entry = bisect.bisect_left(all_spectra, lb)

                # see if the NEXT value is in the range. If so, keep the kmer
                if beginning_entry + 1 < len(all_spectra) and all_spectra[beginning_entry] <= ub:
                    
                    if ion == 'b':
                        b_hits[kmer[:c+1]] = None
                        t.insert(prot_name, kmer[:c+1])
                    else:
                        y_hits[kmer[-c-1:]] = None
                        t.insert(prot_name, kmer[-c-1:])

b_hits = {}
y_hits = {}
plen = len(prots)

t = Tree()

for i, (name, prot_entry) in enumerate(prots.items()):
    
    
    print(f'\rOn protein {i+1}/{plen} [{int((i+1) * 100 / plen)}%]', end='')
    
    for j in range(1, max_len):
        kmer = prot_entry.sequence[:j]
        find_kmer_hits(kmer, name)
    
    for j in range(len(prot_entry.sequence) - max_len):
        kmer = prot_entry.sequence[j:j+max_len]
        find_kmer_hits(kmer, name)     
        
    for j in range(len(prot_entry.sequence) - max_len, len(prot_entry.sequence)):
        kmer = prot_entry.sequence[j:]
        find_kmer_hits(kmer, name)
                    

On protein 279/279 [100%]CPU times: user 17.3 s, sys: 201 ms, total: 17.5 s
Wall time: 17.6 s


## 6. Build the MassDawgs for the different ions

In [9]:
b_dog = PyMassDawg()
y_dog = PyMassDawg()

In [10]:
blen = len(b_hits)
for i, kmer in enumerate(sorted(b_hits.keys(), key=gen_min_ordering)):
    print(f'\rOne kmer {i}/{blen} [{int(100 * (i+1)/blen)}%]', end='')
    bs = gen_spectrum(kmer, ion='b', charge=1)['spectrum']
    bd = gen_spectrum(kmer, ion='b', charge=2)['spectrum']
    b_dog.insert(bs, bd, kmer)
b_dog.finish()

One kmer 1627/28587 [5%]

One kmer 3007/28587 [10%]

One kmer 4126/28587 [14%]

One kmer 5452/28587 [19%]

One kmer 6625/28587 [23%]

...

One kmer 24118/28587 [84%]

One kmer 25482/28587 [89%]

One kmer 26617/28587 [93%]

One kmer 28045/28587 [98%]

One kmer 28586/28587 [100%]

In [11]:
ylen = len(y_hits)
for i, kmer in enumerate(sorted(y_hits.keys(), key=lambda x: gen_min_ordering(x[::-1]))):
    print(f'\rOne kmer {i}/{ylen} [{int(100 * (i+1)/ylen)}%]', end='')
    if 'GYK' == kmer[-3:]:
        print(kmer)
    ys = gen_spectrum(kmer, ion='y', charge=1)['spectrum']
    yd = gen_spectrum(kmer, ion='y', charge=2)['spectrum']
    y_dog.insert(ys, yd, kmer)

y_dog.finish()

One kmer 1184/26679 [4%]

One kmer 2472/26679 [9%]

One kmer 3522/26679 [13%]

One kmer 4811/26679 [18%]

One kmer 5906/26679 [22%]

...

One kmer 21508/26679 [80%]

One kmer 22041/26679 [82%]

One kmer 22507/26679 [84%]

One kmer 23157/26679 [86%]

One kmer 26678/26679 [100%]

In [12]:
db = Database(fasta_file, prots, t, b_dog, y_dog)

## 7. For each spectrum, search the graphs and build an alignment

In [13]:
alignments = {}
gap = 2
for spec_c, spectrum in enumerate(spectra):
    print(f'On spectrum {spec_c + 1}/{len(spectra)} [{int(100 * (spec_c+1)/len(spectra))}%]')
    b_kmers = db.b_dawg.fuzzy_search(spectrum.spectrum, gap, ppm_tol)
    sorted_b_results = sorted(
        [(kmer, scoring.score_subsequence(spectrum.spectrum, kmer, ppm_tol)[0]) for kmer in b_kmers],
         key=lambda x: x[1], reverse=True
    )
    max_score = sorted_b_results[0][1]
    filtered_b_results = [x[0] for x in sorted_b_results if x[1] == max_score]
    
    print(f'\nB results for sequence {spec_c}:\n{filtered_b_results}')

    y_kmers = db.y_dawg.fuzzy_search(spectrum.spectrum, gap, ppm_tol)
    sorted_y_results = sorted(
        [(kmer, scoring.score_subsequence(spectrum.spectrum, kmer, ppm_tol)[1]) for kmer in y_kmers], 
        key=lambda x: x[1], 
        reverse=True
    )
    max_score = sorted_y_results[0][1]
    filtered_y_results = [x[0] for x in sorted_y_results if x[1] == max_score]
    print(f'\nY results for sequence {spec_c}:\n{filtered_y_results}')
    
    alignments[spec_c] = alignment.attempt_alignment(
        spectrum,
        db,
        filtered_b_results,
        filtered_y_results,
        ppm_tolerance=ppm_tol,
        n=3
    )

On spectrum 1/9 [11%]

B results for sequence 0:
['LDDI', 'DLDI', 'PVEGAF', 'EVEV', 'DLDL', 'DIVE', 'DIDI', 'EVVE', 'VEDI', 'IDDI', 'VEDL', 'LDEV', 'VEVE', 'LDID', 'LDDL', 'DLVE', 'DIID', 'LDLD', 'EVLD', 'IDEV', 'IDLD', 'VELD', 'LDVE', 'VEEV', 'DIDL', 'DLEV', 'DILD', 'DLID', 'IDDL', 'KCPQ', 'EVDI']

Y results for sequence 0:
['QWLS', 'DFKG', 'DFGK']
On spectrum 2/9 [22%]

B results for sequence 1:
['SSSY', 'SSPV', 'EGAI', 'EGAD', 'DAAL', 'GEAD', 'DAAD', 'DAAI', 'SSAA', 'ADAD', 'SATG', 'SSYS', 'SASA', 'GEAL', 'ADAL', 'SSVP', 'ADAI', 'SAAS', 'EGAL', 'GEAI', 'SAGT', 'SAPVPVV']

Y results for sequence 1:
['AIVGYK']
On spectrum 3/9 [33%]

B results for sequence 2:
['SLERE', 'TVLS', 'TVEER', 'SSLI', 'SSQPKS', 'SLDKNG', 'SETD', 'SLSL', 'SLEER', 'SILS', 'SSIL', 'SEGGKR', 'TTDD', 'SLTV', 'SINGSR', 'TDTD', 'SLLS', 'TDSE', 'SESE', 'SLIS', 'SSLL', 'TDES', 'SERLE', 'SSEE', 'TVDLW', 'TVVT', 'SIVSNN', 'TDDT', 'TVSI', 'SEDT', 'SEERI', 'SIIS', 'SLSQQA', 'SLFPF', 'SSII', 'SLVT', 'SSGAQAL', 'TVTV', 'SEES

In [14]:
print('Sequence \t | \t scores \t | \t precursor distance')
for i, alignment in alignments.items():
    print(f'Alignments for sequence {i}')
    for a in alignment:
        print(f'{a.sequence} \t | \t b: {a.b_score}, y: {a.y_score} \t | \t {a.precursor_distance}')

Sequence 	 | 	 scores 	 | 	 precursor distance
Alignments for sequence 0
LDDLGNQWLS 	 | 	 b: 2, y: 2 	 | 	 0.9874947499999962
DIIDFGKHKS 	 | 	 b: 2, y: 0 	 | 	 0.5136797500001649
IDLDFGKHKS 	 | 	 b: 2, y: 0 	 | 	 0.5136797500001649
Alignments for sequence 1
DAAIVGYK 	 | 	 b: 3, y: 6 	 | 	 0.0012217500000701875
GEALVGYK 	 | 	 b: 3, y: 6 	 | 	 0.0012217500000701875
ADALVGYK 	 | 	 b: 3, y: 6 	 | 	 0.0012217500000701875
Alignments for sequence 2
Alignments for sequence 3
DPQVEQLEL 	 | 	 b: 6, y: 2 	 | 	 0.000672749999921507
DPQVEQLEN 	 | 	 b: 6, y: 0 	 | 	 0.4787587500001109
PQVEQLENN 	 | 	 b: 1, y: 0 	 | 	 0.013249249999944368
Alignments for sequence 4
DLQTLALWSRM 	 | 	 b: 4, y: 8 	 | 	 0.0001922500000546279
DLQTLAWSRMD 	 | 	 b: 5, y: 0 	 | 	 0.9716317500000287
Alignments for sequence 5
SLEEREHMDVPSV 	 | 	 b: 3, y: 3 	 | 	 0.02964724999981172
DPVLTVEEMDVPSV 	 | 	 b: 1, y: 3 	 | 	 0.985137750000149
KDLTEYMMDVPSV 	 | 	 b: 1, y: 3 	 | 	 0.03157074999978704
Alignments for sequence 6
ELTNIELL 

In [15]:
print(spectra[1].spectrum)
print(sorted(gen_spectrum('VGYK', ion='y', charge=1)['spectrum']))
print(sorted(gen_spectrum('VGYK', ion='y', charge=2)['spectrum']))

[72.08074188232422, 86.0962142944336, 88.0391616821289, 99.05437469482422, 120.08078002929688, 129.10186767578125, 136.074462890625, 141.06512451171875, 142.05029296875, 147.11251831054688, 157.13116455078125, 159.0762939453125, 187.0716094970703, 213.0857391357422, 258.1067810058594, 270.8402099609375, 310.1756591796875, 367.1947326660156, 371.1916809082031, 406.25396728515625, 466.2637634277344, 467.26953125, 579.3480834960938, 650.3857421875, 651.3853149414062]
[147.112804135, 310.176124135, 367.197588135, 466.266002135]
[74.060040285, 155.591700285, 184.102432285, 233.636639285]


In [16]:
da_tol = ppm_to_da(310.176124135, 20)

In [17]:
sorted(db.y_dawg.fuzzy_search(spectra[1].spectrum, 0, 20))

['AATKFI',
 'AFTLKA',
 'AIVGYK',
 'ALTFAK',
 'DFKIK',
 'DKFKL',
 'DKFLK',
 'FKKDL',
 'FSTEV',
 'FSVKAV',
 'GFKVSI',
 'GKFVLS',
 'IKQVY',
 'IKQYV',
 'IKSVFG',
 'IVQKY',
 'KDIVS',
 'KDKFL',
 'KFEKV',
 'KFEVK',
 'KFKVE',
 'KFVLSG',
 'KGSIFV',
 'KIFDK',
 'KKFDL',
 'KKFLD',
 'KLVQY',
 'KYLLN',
 'LAAFTK',
 'LAKTAF',
 'LFKDK',
 'LKYVQ',
 'LLNYK',
 'LLPPPN',
 'LPQPVP',
 'LVQYK',
 'MKIFD',
 'MSVFGK',
 'MYKLL',
 'NIYKL',
 'NKYLL',
 'NLPLPP',
 'NNPELA',
 'NYLIK',
 'PELLN',
 'PSPVAPA',
 'PVAPAPV',
 'PVPAAPV',
 'PVPPLQ',
 'SVFGKL',
 'VAPAPVP',
 'VAPPPAV',
 'VLQKY',
 'VPAAPVP',
 'VPSPVAP',
 'VQLYK',
 'YGVLKA',
 'YKLLN',
 'YLAVKG',
 'YLIKN',
 'YLKNL',
 'YLKQV',
 'YLLKN',
 'YLTIP',
 'YVIQK']